In [1]:
import pandas as pd
import requests
import datetime as dt
# import numpy as np
import smtplib
from bs4 import BeautifulSoup

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
# import sys
# from IPython.display import HTML

In [2]:
pd.set_option('display.max_columns', None)


Yield Curve

In [ ]:
# US Treasury website link containing yield curve data
url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value_month="+str(dt.date.today().year)+str(dt.date.today().month)

# http request 
response = requests.get(url)
# parsing response html
doc=BeautifulSoup(response.text,"html.parser")
# finding all the "td" elememts
div_elements=doc.findAll('td')
# creating a dataframe to store data
df=pd.DataFrame(columns=('Date','Header','Value'))
# iterating through each "td" element 
for x in div_elements:   
    # if Date value found store the date in temp_date variable
    if x.attrs['class'][1]=='views-field-field-tdr-date'  :
        temp_date=x.find('time')
    # Storing row data to dataframe 
    df.loc[df.shape[0]+1]=(temp_date.text,x.attrs['headers'][0],x.text)
#converting date format to datetime format

df['Date']=pd.to_datetime(df['Date'])
# filtering the data for current date
df[df['Date']==str(dt.date.today())]

# df[df['Date']==str(dt.date(year=2023,month=12,day=15))]

# todays_data=df[df['Date']==str(dt.date(year=2023,month=12,day=15))]
# pivoting the data frame
pivoted_df=todays_data.pivot(index='Date',columns='Header',values='Value')
# removing empty columns
pivoted_df=pivoted_df.iloc[:,0:13]

# removing extra characters from column names
New_names=[]
for name in pivoted_df.columns.tolist():
    temp_name=name.replace('view-field-bc-','')
    temp_name=temp_name.replace('-table-column','')
    temp_name=temp_name.replace('view-field-br-round-b1','')
    temp_name=temp_name.replace('view-field-tc-','')
    temp_name=temp_name.replace('view-field-tc-','')

    New_names.append(temp_name)
pivoted_df.columns=New_names

# resetting index so date column can be a column
pivoted_df.reset_index(inplace=True)
# converting data to float
pivoted_df[['10year','3month','2year']]=pivoted_df[['10year','3month','2year']].astype('float')

# calculating Yield curve ratio based on arturo method 
pivoted_df['YC Ratio arturo']=pivoted_df['10year']-pivoted_df['3month']
# calculating Yield curve ratio based on wallstreet method 
pivoted_df['YC Ratio Wallstreet']=pivoted_df['10year']-pivoted_df['2year']


Stock Price Notification

In [ ]:
# yahoo finance api
url='https://yfapi.net/v6/finance/quote?region=US&lang=en&symbols='
# header with api key 
headers = {'X-API-KEY': 'xzxxxxxxx'}
# accesing stock datasheet from google drive
stock_list=pd.read_csv('xxxxxxxxxxxxxxxxx')

Request_list=''
temp_arr=[]
i=0

# iterating throug  stock list and creating a array with 10 row each (due to yahoo api limiation)
for n in stock_list['Symbol']:
        i=i+1
        if i<=10:
            if len(Request_list)==0:
                Request_list=n
            else:
                Request_list=Request_list+','+n  
        else:
            i=0
            temp_arr.append(Request_list)            
            del(Request_list)
            Request_list=''
            Request_list=n
if len(Request_list.split(','))<=10:
    temp_arr.append(Request_list)

# iterating through the stock list to get the latest stock price.
for items in temp_arr:
    request_url=url+items
    response=requests.get(request_url,headers=headers)
    
    response_json=response.json()
    for item in response_json['quoteResponse']['result']:
      stock_list.loc[stock_list['Symbol']==item['symbol'],('Description','Last Price')]=(item['longName'],item['regularMarketPrice'])

# checking if the stock price is high or lower than the notify price set in google sheet
for i,r in stock_list.iterrows():
    if (r['Type']=='S'):
        if r['Last Price']>=r['Notify Price']:
           stock_list.loc[i,'Notify']=True  
        else:
            stock_list.loc[i,'Notify']=False
    else:
        if r['Last Price']<=r['Notify Price']:
           stock_list.loc[i,'Notify']=True
        else:
            stock_list.loc[i,'Notify']=False

stock_list['Delta']=stock_list['Purchased Price']-stock_list['Last Price']

In [ ]:
# procedure to send email 
def send_email(YC_tbl,Sock_tbl):
  s = smtplib.SMTP('smtp.gmail.com', 587)
  s.starttls()
  s.ehlo()
  s.login("email@email.com", "xxxhhhh")
  # print('Awesome')

  html = """\
  <html>
    <head></head>
    <body>
      <font size="5"><strong>Stock Notification</strong></font>
      <br>
      <br>
      {0}
      <br>
      <br>
      {1}
    </body>
  </html>
  
  """.format(Sock_tbl,YC_tbl)
      
  recipients="two@two.com,one@one.com"

  SUBJECT="Stock Price Changed"
  TEXT = "This message was sent with Python's smtplib."

  part1 = MIMEText(html, 'html')

  message = MIMEMultipart()
  message.attach(part1)
  message['Subject'] = SUBJECT
  message['To']=recipients

  s.sendmail("email@email.com", recipients.split(","), message.as_string())

  s.quit()

In [ ]:
# if Yield curve is less than 0.1 then notify user 
if pivoted_df['YC Ratio arturo'].values[0]<0.1:
  YC_html_table=pivoted_df.to_html()
  YC_html_table=" <br>      <br>      US Treasury Yield Curve Rates      <br>"+YC_html_table
else:
  YC_html_table=''  
  
# if stock price is higher than notify price then send email to  user  
if len(stock_list[stock_list['Notify']==True])!=0:
  stock_html_table=stock_list[stock_list['Notify']==True].to_html()
  stock_html_table=" <br>      <br>   List of Stocks Price Changed <br>"+stock_html_table
else:
      stock_html_table='' 
  
# send email to user with stock price and Yield curve information
if pivoted_df['YC Ratio arturo'].values[0]<0.1 or len(stock_list[stock_list['Notify']==True])!=0:
   send_email(YC_html_table,stock_html_table)